# Machine Learning Fairness

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).

**Fairness** in [machine learning](https://en.wikipedia.org/wiki/Machine_Learning "Machine Learning") refers to the various attempts at correcting [algorithmic bias](https://en.wikipedia.org/wiki/Algorithmic_bias "Algorithmic bias") in automated decision processes based on machine learning models. Decisions made by computers after a machine-learning process may be considered unfair if they were based on [variables](https://en.wikipedia.org/wiki/Dependent_and_independent_variables "Dependent and independent variables") considered **sensitive**. Examples of these kinds of variable include [gender](https://en.wikipedia.org/wiki/Gender "Gender"), [ethnicity](https://en.wikipedia.org/wiki/Ethnicity), [sexual orientation](https://en.wikipedia.org/wiki/Sexual_orientation "Sexual orientation"), [disability](https://en.wikipedia.org/wiki/Disability "Disability") and more.

As is the case with many ethical concepts, definitions of fairness and bias are always controversial. In general, fairness and bias are considered relevant when the decision process impacts people's lives. In [machine learning](https://en.wikipedia.org/wiki/Machine_learning "Machine learning"), the problem of [algorithmic bias](https://en.wikipedia.org/wiki/Algorithmic_bias "Algorithmic bias") is well known and well studied. Outcomes may be skewed by a range of factors and thus might be considered unfair concerning certain groups or individuals.

![image](https://repository-images.githubusercontent.com/250897484/9466ef80-cb69-11ea-9bc7-404fd8b3bddc)


In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import Markdown
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

### Credit Approval Data Set

The data set used is this notebook is the [**Credit Approval Data Set**](https://archive.ics.uci.edu/ml/datasets/credit+approval), made available by the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). This dataset contains labeled samples (approved/unapproved) of credit card applications.


In [ ]:
data = pd.read_csv(r'data\dataCC.csv', header=0)
data.columns = ['Gender', 'Age', 'Debt', 'Married', 'Bank Client', 'Education',
                'Race', 'Years Employed', 'Prior Default', 'Employed', 'Credit',
                "Driver's License", 'Citizenship', 'Postal code', 'Income', 'Approval Status']

data = data.replace('?', np.nan)
data.fillna(data.mean(), inplace=True)
for col in data.columns:
    if data[col].dtypes == 'object':
        data = data.fillna(data[col].value_counts().index[0])

data = data.drop(['Age', 'Married', 'Bank Client', 'Education',
                  'Race', 'Years Employed', 'Employed', 'Credit',
                  "Driver's License", 'Citizenship', 'Postal code', 'Income'], axis=1)

data['Gender'].replace({'b': 'Male', 'a': 'Female'}, inplace=True)
data['Approval Status'].replace({'+': 1, '-': 0}, inplace=True)
data['Prior Default'].replace({'t': 1, 'f': 0}, inplace=True)


## FACETS

[Facets](https://pair-code.github.io/facets/) its a lybrary for data visualization. It contains two robust visualizations to aid in understanding and analyzing machine learning datasets. Get a sense of the shape of each feature of your dataset using Facets Overview, or explore individual observations using Facets Dive.

![facets](https://3.bp.blogspot.com/-T0dTxdse9Ow/WWz0u431RpI/AAAAAAAAB5M/rBvToJjx1L0FVVpXkgNOAwzXASyZC_JWwCLcBGAs/s640/image4.gif)


In [3]:
from facets_overview.feature_statistics_generator import FeatureStatisticsGenerator
from IPython.display import HTML
import base64

fsg = FeatureStatisticsGenerator()
dataframes = [
    {'table': data, 'name': 'trainData'}]
censusProto = fsg.ProtoFromDataFrames(dataframes)
protostr = base64.b64encode(censusProto.SerializeToString()).decode('utf-8')


HTML_TEMPLATE = '''<script src='https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/1.3.3/webcomponents-lite.js'></script>
        <link rel='import' href='https://raw.githubusercontent.com/PAIR-code/facets/1.0.0/facets-dist/facets-jupyter.html'>
        <facets-overview id='elem'></facets-overview>
        <script>
          document.querySelector('#elem').protoInput = '{protostr}';
        </script>'''
html = HTML_TEMPLATE.format(protostr=protostr)
display(HTML(html))
with open("overview_data_credit_card.html", "w") as file:
    file.write(html)
display(HTML("<a href='overview_data_credit_card.html' target='_blank'>overview_data_credit_card.html</a>"))


In [4]:
SAMPLE_SIZE = 500

data_dive = data.sample(SAMPLE_SIZE).to_json(orient='records')

HTML_TEMPLATE = """<script src="https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/1.3.3/webcomponents-lite.js"></script>
        <link rel="import" href="https://raw.githubusercontent.com/PAIR-code/facets/1.0.0/facets-dist/facets-jupyter.html">
        <facets-dive id="elem" height="600"></facets-dive>
        <script>
          var data = {jsonstr};
          document.querySelector("#elem").data = data;
        </script>"""
html = HTML_TEMPLATE.format(jsonstr=data_dive)

print('''
For some reason, the Dive HTML does not open well in VScode notebooks, 
but you can see the dash following the link below:
''')
with open("dive_data_credit_card.html", "w") as file:
    file.write(html)
display(HTML("<a href='dive_data_credit_card.html' target='_blank'>dive_data_credit_card.html</a>"))



For some reason, the Dive HTML does not open well in VScode notebooks, 
but you can see the dash following the link below:



## Pandas Profiling

![pandas-profiling](https://warehouse-camo.ingress.cmh1.psfhosted.org/6d498300bf33179bd2299e521adf386991ac9ba4/68747470733a2f2f70616e6461732d70726f66696c696e672e79646174612e61692f646f63732f6173736574732f6c6f676f5f6865616465722e706e67)

**Another way to find information about a `pd.DataFrame`, besides using `facets` and functions like `describe` and `info`, is by using the `pandas-profiling` [module](https://pypi.org/project/pandas-profiling/).**

**`pandas-profiling` generates profile reports from a pandas `DataFrame`. Extending a pandas `DataFrame` with `df.profile_report()`, will automatically generate a standardized univariate and multivariate report for data understanding.**

In [ ]:
from pandas_profiling import ProfileReport

profile = ProfileReport(data, title="Pandas Profiling Report")
profile.to_notebook_iframe()
profile.to_file("pandas_profiling_credit_card.html")

## Training a ML model with `biased` data


In [5]:
seed = 42

train, test = \
    train_test_split(
        data, stratify=data['Gender'], test_size=0.2, random_state=seed)

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)


lr = LogisticRegression(random_state=seed)
lr.fit(train[['Debt', 'Prior Default']], train['Approval Status'])
score = lr.score(test[['Debt', 'Prior Default']], test['Approval Status'])
preds = lr.predict(test[['Debt', 'Prior Default']])
matrix = confusion_matrix(test['Approval Status'], preds).ravel()
print(f'Accuracy: ' + '{:.2f}'.format(score * 100) + ' %')


Accuracy: 87.68 %


In [6]:
fig = px.imshow(matrix.reshape(2, 2),
                labels=dict(x="Predicted", y="True label"),
                x=['Approved', 'Not approved'],
                y=['Approved', 'Not approved'],
                text_auto=True
                )
fig.update_xaxes(side='top')
fig.update_layout(template='plotly_dark',
                  title='Confusion Matrix',
                  coloraxis_showscale=False,
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)')
fig.show()


### Statistical Fairness Metrics

**_Fairness metrics are a set of measures that enable you to detect the presence of bias in your data or model._ For a full review of the most prominent definitions of fairness in algorithmic classification, we recommend [_Fairness Definitions Explained_](https://fairware.cs.umass.edu/papers/Verma.pdf). Below you will see how to apply these definitions in code.**

**Confusion Matrix cheatsheet:**

![confusion-matrix](https://ars.els-cdn.com/content/image/3-s2.0-B9780128241455000058-f03-05-9780128241455.jpg)

- $TP$: True Positive
- $FP$: False Positive
- $TN$: True Neagative
- $FN$: False Negative

### Common ML Fairness Metrics

**Statistical Parity Ratio: _Statistical Parity Ratio_ compares the proportion of members of a given group that were classified for the positive class (i.e., correctly or not, a.k.a., TP and FP) to another group (privileged $\times$ unprivileged).**

$$\frac{Statistical\;Parity_{\;unprivileged}}{Statistical\;Parity_{\;privileged}} = \frac{(\frac{TP  +  FP}{TP  +  FP  +  TN  +  FN})^{unprivileged}}{(\frac{TP  +  FP}{TP  +  FP  +  TN  +  FN})^{privileged}}$$

**Equal Opportunity Ratio: _Equal Opportunity ratio_ compares the true positive rate (i.e., TPR, a.k.a., _Sensitivity/Recall_) of different groups (privileged $\times$ unprivileged).**

$$\frac{TPR_{\;unprivileged}}{TPR_{\;privileged}} = \frac{(\frac{TP}{TP+FN})^{unprivileged}}{(\frac{TP}{TP+FN})^{privileged}}$$

**Predictive Parity Ratio: _Predictive Parity Ratio_ compares the positive predictive value (i.e., PPV, a.k.a., _Precision_) of different groups (privileged $\times$ unprivileged).**

$$\frac{PPV_{\;unprivileged}}{PPV_{\;privileged}} = \frac{(\frac{TP}{TP+FP})^{unprivileged}}{(\frac{TP}{TP+FP})^{privileged}}$$

**Predictive Equality Ratio: _Predictive Equality Ratio_ compares the false positive rate (FPR, a.k.a., _fall-out/false alarm ratio_) of different groups (privileged $\times$ unprivileged).**

$$\frac{FPR_{\;unprivileged}}{FPR_{\;privileged}} = \frac{(\frac{FP}{FP+TN})^{unprivileged}}{(\frac{FP}{FP+TN})^{privileged}}$$

**Accuracy Equality Ratio: _Accuracy Equality Ratio_ compares the proportion of members of a given group that were correctly classified (i.e., _accuracy_) to another group (privileged $\times$ unprivileged).**

$$\frac{Accuracy_{\;unprivileged}}{Accuracy_{\;privileged}} = \frac{(\frac{TP  +  TN}{TP  +  FP  +  TN  +  FN})^{unprivileged}}{(\frac{TP  +  TN}{TP  +  FP  +  TN  +  FN})^{privileged}}$$

**Equalized Odds: _Equalized Odds_ it is the most restrictive concept of ML Fairness. This criteria is only satisfied if both groups (privileged $\times$ unprivileged) have equal TPR and FPR.**

$$\frac{TPR_{\;unprivileged}}{TPR_{\;privileged}} = 1 \;\land\;\frac{FPR_{\;unprivileged}}{FPR_{\;privileged}} = 1$$


In [45]:
def calc_fair(model, DataFrame, protected_atributte, group_priv, group_unpriv, label):
    test_set = DataFrame

    test_set_priv_labels, test_set_priv = list(test_set[test_set[protected_atributte] == group_priv][label]), test_set[test_set[protected_atributte] == group_priv].drop([label, protected_atributte], axis = 1)
    test_set_unpriv_labels, test_set_unpriv = list(test_set[test_set[protected_atributte] == group_unpriv][label]), test_set[test_set[protected_atributte] == group_unpriv].drop([label, protected_atributte], axis = 1)
    
    preds_priv = model.predict(test_set_priv)
    preds_unpriv = model.predict(test_set_unpriv)

    TN_PV, FP_PV, FN_PV, TP_PV = confusion_matrix(test_set_priv_labels, preds_priv).ravel()
    TN_UPV, FP_UPV, FN_UPV, TP_UPV = confusion_matrix(test_set_unpriv_labels, preds_unpriv).ravel()

    statistical_parity_priv = (TP_PV + FP_PV)/(TP_PV + FP_PV + TN_PV + FN_PV)  # STATISTICAL PARITY RATIO
    statistical_parity_unpriv = (TP_UPV + FP_UPV)/(TP_UPV + FP_UPV + TN_UPV + FN_UPV)  # STATISTICAL PARITY RATIO
    equal_oportunity_priv = TP_PV / (TP_PV+FN_PV)  # TRUE POSITIVE RATIO
    equal_oportunity_unpriv = TP_UPV / (TP_UPV+FN_UPV)  # TRUE POSITIVE RATIO
    predictive_parity_priv = TP_PV/(TP_PV + FP_PV)  # POSITIVE PREDICTIVE VALUE
    predictive_parity_unpriv = TP_UPV/(TP_UPV + FP_UPV)  # POSITIVE PREDICTIVE VALUE
    predictive_equality_priv = FP_PV / (FP_PV+TN_PV)  # FALSE POSITIVE RATE
    predictive_equality_unpriv = FP_UPV / (FP_UPV+TN_UPV)  # FALSE POSITIVE RATE
    accuracy_equality_priv = (TP_PV + TN_PV)/(TP_PV + FP_PV + TN_PV + FN_PV)  # ACCURACY EQUALITY RATIO
    accuracy_equality_unpriv = (TP_UPV + TN_UPV)/(TP_UPV + FP_UPV + TN_UPV + FN_UPV)  # ACCURACY EQUALITY RATIO 

    if statistical_parity_priv >= statistical_parity_unpriv:
        statistical_parity_ratio = statistical_parity_unpriv/statistical_parity_priv
    elif statistical_parity_priv < statistical_parity_unpriv:
        statistical_parity_ratio = statistical_parity_priv/statistical_parity_unpriv

    if equal_oportunity_priv >= equal_oportunity_unpriv:
        equal_oportunity_ratio = equal_oportunity_unpriv/equal_oportunity_priv
    elif equal_oportunity_priv < equal_oportunity_unpriv:
        equal_oportunity_ratio = equal_oportunity_priv/equal_oportunity_unpriv
    
    if predictive_parity_priv >= predictive_parity_unpriv:
        predictive_parity_ratio = predictive_parity_unpriv/predictive_parity_priv
    elif predictive_parity_priv < predictive_parity_unpriv:
        predictive_parity_ratio = predictive_parity_priv/predictive_parity_unpriv

    if predictive_equality_priv >= predictive_equality_unpriv:
        predictive_equality_ratio = predictive_equality_unpriv/predictive_equality_priv
    elif predictive_equality_priv < predictive_equality_unpriv:
        predictive_equality_ratio = predictive_equality_priv/predictive_equality_unpriv
    
    if accuracy_equality_priv >= accuracy_equality_unpriv:
        accuracy_equality_ratio = accuracy_equality_unpriv/accuracy_equality_priv
    elif accuracy_equality_priv < accuracy_equality_unpriv:
        accuracy_equality_ratio = accuracy_equality_priv/accuracy_equality_unpriv

    equalized_odds = f'TPR: {round(equal_oportunity_priv, 2)} vs {round(equal_oportunity_unpriv, 2)} <br> FPR: {round(predictive_equality_priv,2)} vs {round(predictive_equality_unpriv,2)}'

    data = {'Fairness Metrics': ['Chance of receiving the positive class - privileged',
                                'Chance of receiving the positive class - unprivileged',
                                'Statistical Parity Ratio (SPR)',
                                'True Positive Rate - privileged',
                                'True Positive Rate - unprivileged',
                                'Equal Opportunity Ratio (EOR)',
                                'Positive Predictive Value - privileged',
                                'Positive Predictive Value - unprivileged',
                                'Predictive Parity Ratio (PPR)',
                                'False Positive Rate - privileged',
                                'False Positive Rate - unprivileged',
                                'Predictive Equality Ratio (PER)',
                                'Accuracy - privileged',
                                'Accuracy - unprivileged',
                                'Accuracy Equality Ratio (AER)',
                                'Equalized Odds'],
            'Scores': [round(statistical_parity_priv, 2),
                        round(statistical_parity_unpriv, 2),
                        round(statistical_parity_ratio,2),
                        round(equal_oportunity_priv, 2),
                        round(equal_oportunity_unpriv, 2),
                        round(equal_oportunity_ratio, 2),
                        round(predictive_parity_priv,2),
                        round(predictive_parity_unpriv,2),
                        round(predictive_parity_ratio,2),
                        round(predictive_equality_priv,2),
                        round(predictive_equality_unpriv,2),
                        round(predictive_equality_ratio,2),
                        round(accuracy_equality_priv,2),
                        round(accuracy_equality_unpriv,2),
                        round(accuracy_equality_ratio,2),
                        f'TPR: {round(equal_oportunity_priv, 2)} vs {round(equal_oportunity_unpriv, 2)}. FPR: {round(predictive_equality_priv,2)} vs {round(predictive_equality_unpriv,2)}']
            }
    df = pd.DataFrame(data).set_index('Fairness Metrics')

    return df, display(Markdown(f'''
|Fairness Scores | Score |
|--|--|
| Chance of receiving the positive class - privileged |{round(statistical_parity_priv, 2)}|
| Chance of receiving the positive class - unprivileged |{round(statistical_parity_unpriv, 2)}|
| Statistical Parity Ratio (SPR)|{round(statistical_parity_ratio,2)}|
| True Positive Rate - privileged |{round(equal_oportunity_priv, 2)}|
| True Positive Rate - unprivileged |{round(equal_oportunity_unpriv, 2)}|
| Equal Opportunity Ratio (EOR)|{round(equal_oportunity_ratio, 2)}|
| Positive Predictive Value - privileged |{round(predictive_parity_priv,2)}|
| Positive Predictive Value - unprivileged |{round(predictive_parity_unpriv,2)}|
| Predictive Parity Ratio (PPR)|{round(predictive_parity_ratio,2)}|
| False Positive Rate - privileged |{round(predictive_equality_priv,2)}|
| False Positive Rate - unprivileged |{round(predictive_equality_unpriv,2)}|
| Predictive Equality Ratio (PER)|{round(predictive_equality_ratio,2)}|
| Accuracy - privileged |{round(accuracy_equality_priv,2)}|
| Accuracy - unprivileged |{round(accuracy_equality_unpriv,2)}|
| Accuracy Equality Ratio (AER)|{round(accuracy_equality_ratio,2)}|
| Equalized Odds|{equalized_odds}|

'''
))



In [57]:
results_df, results_md = calc_fair(lr, test, 'Gender', 'Female', 'Male', 'Approval Status')
display(results_df)


|Fairness Scores | Score |
|--|--|
| Chance of receiving the positive class - privileged |0.5|
| Chance of receiving the positive class - unprivileged |0.5|
| Statistical Parity Ratio (SPR)|1.0|
| True Positive Rate - privileged |0.86|
| True Positive Rate - unprivileged |0.89|
| Equal Oportunity Ratio (EOR)|0.96|
| Positive Predictive Value - privileged |0.86|
| Positive Predictive Value - unprivileged |0.88|
| Predictive Parity Ratio (PPR)|0.98|
| False Positive Rate - privileged |0.14|
| False Positive Rate - unprivileged |0.12|
| Predictive Equality Ratio (PER)|0.86|
| Accuracy - privileged |0.86|
| Accuracy - unprivileged |0.89|
| Accuracy Equality Ratio (AER)|0.97|
| Equalized Odds|TPR: 0.86 vs 0.89 <br> FPR: 0.14 vs 0.12|



,Scores
Fairness Metrics,
Chance of receiving the positive class - privileged,0.5
Chance of receiving the positive class - unprivileged,0.5
Statistical Parity Ratio (SPR),1.0
True Positive Rate - privileged,0.86
True Positive Rate - unprivileged,0.89
Equal Oportunity Ratio (EOR),0.96
Positive Predictive Value - privileged,0.86
Positive Predictive Value - unprivileged,0.88
Predictive Parity Ratio (PPR),0.98


**Fairness metrics can help us find out if our model is discriminating according to any particular definition of fairness. However, as an [impossibility theorem](https://arxiv.org/abs/2007.06024), we cannot satisfy all Fairness metrics (SP, EO, PP) together. Thus, the choice of which metric to use must be made according to the context of an application (i.e., benefit awarding, medical diagnosis, etc.).**

---

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).
